# 2022.09.05.(화)
---
##### [1] language 
- 목적    : 알파벳(a-z)을 사용하는 언어 식별
- 데이터  : 알파벳 사용하는 4개국의 테스트 파일
- 전제조건: 나라별 자주 사용되는 알파벳이 다름

(1) 전처리   
- 나라별 알파벳 패턴 분석  
  
(2) 학습   
- 교차 검증 적용  

In [1]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.utils import all_estimators
from sklearn.model_selection import train_test_split

path_train = './lang/train/'
path_test  = './lang/test/'

# 폴더 내 모든 파일 불러오기
dir_list = os.listdir(path_train)
dir_list

['en-1.txt',
 'en-2.txt',
 'en-3.txt',
 'en-4.txt',
 'en-5.txt',
 'fr-10.txt',
 'fr-6.txt',
 'fr-7.txt',
 'fr-8.txt',
 'fr-9.txt',
 'id-11.txt',
 'id-12.txt',
 'id-13.txt',
 'id-14.txt',
 'id-15.txt',
 'tl-16.txt',
 'tl-17.txt',
 'tl-18.txt',
 'tl-19.txt',
 'tl-20.txt']

In [2]:
with open(path_train + 'en-1.txt', mode ='r', encoding='utf-8') as f:
    text = f.read()

# 모두 소문자로 변경    
text = text.lower()
text

'\n\n\n\nthe main henry ford museum building houses some of the classrooms for the henry ford academy\n\n\nhenry ford academy is the first charter school in the united states to be developed jointly by a global corporation, public education, and a major nonprofit cultural institution. the school is sponsored by the ford motor company, wayne county regional educational service agency and the henry ford museum and admits high school students. it is located in dearborn, michigan on the campus of the henry ford museum. enrollment is taken from a lottery in the area and totaled 467 in 2010.[1]\nfreshman meet inside the main museum building in glass walled classrooms, while older students use a converted carousel building and pullman cars on a siding of the greenfield village railroad. classes are expected to include use of the museum artifacts, a tradition of the original village schools. when the museum was established in 1929, it included a school which served grades kindergarten to colle

In [3]:
# 정규식으로 알파벳만 추출
import re
p = re.compile('[a-z]+')
alphabet_text_list = p.findall(text)
#print(alphabet_text_list)

# 리스트 -> 문자열로 변환
alphabet_text =' '.join(s for s in alphabet_text_list)
print(alphabet_text)

the main henry ford museum building houses some of the classrooms for the henry ford academy henry ford academy is the first charter school in the united states to be developed jointly by a global corporation public education and a major nonprofit cultural institution the school is sponsored by the ford motor company wayne county regional educational service agency and the henry ford museum and admits high school students it is located in dearborn michigan on the campus of the henry ford museum enrollment is taken from a lottery in the area and totaled in freshman meet inside the main museum building in glass walled classrooms while older students use a converted carousel building and pullman cars on a siding of the greenfield village railroad classes are expected to include use of the museum artifacts a tradition of the original village schools when the museum was established in it included a school which served grades kindergarten to college trade school ages the last part of the ori

In [4]:
# 알파벳 개수 세기
alphabet_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

for i in range(len(alphabet_list)):
    print(f'{alphabet_list[i]}의 갯수 : {alphabet_text.count(alphabet_list[i])}개')


a의 갯수 : 349개
b의 갯수 : 59개
c의 갯수 : 210개
d의 갯수 : 212개
e의 갯수 : 484개
f의 갯수 : 72개
g의 갯수 : 88개
h의 갯수 : 201개
i의 갯수 : 340개
j의 갯수 : 8개
k의 갯수 : 25개
l의 갯수 : 247개
m의 갯수 : 121개
n의 갯수 : 356개
o의 갯수 : 412개
p의 갯수 : 76개
q의 갯수 : 0개
r의 갯수 : 357개
s의 갯수 : 282개
t의 갯수 : 370개
u의 갯수 : 119개
v의 갯수 : 45개
w의 갯수 : 65개
x의 갯수 : 3개
y의 갯수 : 92개
z의 갯수 : 2개


In [5]:
#for file in dir_list:

def count_alphabet(file_name,path):

    # 무슨 파일인지 명시
    # print(file_name)
    
    # 파일 불러오기
    with open(path + file_name, mode ='r', encoding='utf-8') as f:
        text = f.read()

    # 모두 소문자로 변경    
    text = text.lower()

    # 정규식으로 알파벳만 추출
    p = re.compile('[a-z]+')
    alphabet_text_list = p.findall(text)
    #print(alphabet_text_list)

    # 리스트 -> 문자열로 변환
    alphabet_text =' '.join(s for s in alphabet_text_list)

    # 알파벳 개수 세기
    alphabet_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

    col = []
    for i in range(len(alphabet_list)):
        #print(f'{alphabet_list[i]}의 갯수 : {alphabet_text.count(alphabet_list[i])}개')
        col.append((alphabet_text.count(alphabet_list[i])))
        
    return(col)
    # 데이터프레임으로 만들기


In [6]:
# 각 파일의 알파벳 개수를 하나의 컬럼으로 만들기 -> 데이터프레임으로 생성
df=pd.DataFrame(columns = dir_list) 
for file_name in dir_list:
    col = count_alphabet(file_name,path_train)
    df[file_name] = col
df = df.T
df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
en-1.txt,349,59,210,212,484,72,88,201,340,8,...,0,357,282,370,119,45,65,3,92,2
en-2.txt,706,167,255,326,1149,146,262,230,632,22,...,46,756,601,652,257,115,117,17,90,5
en-3.txt,259,44,165,118,434,53,91,85,342,9,...,6,195,318,293,105,68,43,2,65,2
en-4.txt,383,147,159,210,642,89,125,313,346,8,...,2,314,389,497,129,27,104,32,93,9
en-5.txt,337,93,142,181,645,93,93,260,297,15,...,2,331,272,436,114,49,109,14,68,3
fr-10.txt,1228,163,581,831,2347,253,167,242,1082,36,...,92,1171,1117,1210,834,223,11,61,57,13
fr-6.txt,2162,414,992,1244,4155,327,319,350,1998,59,...,190,2074,2278,1958,1511,295,126,108,148,13
fr-7.txt,1230,221,599,761,2550,178,179,195,1208,51,...,112,1301,1325,1111,861,215,44,84,82,2
fr-8.txt,585,127,321,412,1266,105,109,116,690,35,...,47,661,536,492,344,122,6,41,40,16
fr-9.txt,405,59,181,266,735,62,103,100,461,20,...,24,399,371,325,225,72,2,17,27,6


In [7]:
target = ["en"] * 5 + ["fr"] * 5 + ['id'] *5 +['tl']*5
target

['en',
 'en',
 'en',
 'en',
 'en',
 'fr',
 'fr',
 'fr',
 'fr',
 'fr',
 'id',
 'id',
 'id',
 'id',
 'id',
 'tl',
 'tl',
 'tl',
 'tl',
 'tl']

In [8]:
df.values

array([[  349,    59,   210,   212,   484,    72,    88,   201,   340,
            8,    25,   247,   121,   356,   412,    76,     0,   357,
          282,   370,   119,    45,    65,     3,    92,     2],
       [  706,   167,   255,   326,  1149,   146,   262,   230,   632,
           22,    82,   355,   203,   449,   577,   170,    46,   756,
          601,   652,   257,   115,   117,    17,    90,     5],
       [  259,    44,   165,   118,   434,    53,    91,    85,   342,
            9,     6,   206,    81,   273,   277,    75,     6,   195,
          318,   293,   105,    68,    43,     2,    65,     2],
       [  383,   147,   159,   210,   642,    89,   125,   313,   346,
            8,    69,   238,   114,   311,   465,    89,     2,   314,
          389,   497,   129,    27,   104,    32,    93,     9],
       [  337,    93,   142,   181,   645,    93,    93,   260,   297,
           15,    27,   181,   132,   284,   302,    86,     2,   331,
          272,   436,   114,  

# 분석
---

##### [1] 학습 데이터, 테스트 데이터, 검증용 데이트 분리 

In [9]:
# 학습 - 테스트 데이터셋
train_input, test_input, train_target, test_target =train_test_split(df.values,target,test_size = 0.2, random_state = 42)

##### [2] 학습

In [10]:
# 필터 타입에 해당하는 sklearn에 존재하는 모든 모델 이름과 객체 리스트로 반환
models = all_estimators(type_filter = 'classifier')

# 각 모델들 훈련시키고 정확도 추출
scores = []
for name, model in models:
    try:
        # 모델 객체 생성
        md = model()
        # 학습
        md.fit(train_input, train_target)
        # 평가
        result = md.score(test_input, test_target)
    
        scores.append((name, result))
    except:
        pass

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/semi_supervised/_label_propagation.py:259: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  classes = classes[classes != -1]
/opt/conda/lib/python3.10/site-packages/sklearn/semi_supervised/_label_propagation.py:273: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  unlabeled = y == -1
/opt/conda/lib/python3.10/site-packages/sklearn/semi_supervised/_label_propagation.py:259: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  classes = classes[classes != -1]
/opt/conda/lib/python3.10/site-packages/sklearn/semi_supervised/_l

In [11]:
# 각 분류기 모듈명과 score
scores

[('AdaBoostClassifier', 0.25),
 ('BaggingClassifier', 0.25),
 ('BernoulliNB', 0.0),
 ('CalibratedClassifierCV', 0.5),
 ('CategoricalNB', 0.0),
 ('ComplementNB', 0.5),
 ('DecisionTreeClassifier', 0.25),
 ('DummyClassifier', 0.0),
 ('ExtraTreeClassifier', 0.5),
 ('ExtraTreesClassifier', 0.5),
 ('GaussianNB', 0.25),
 ('GaussianProcessClassifier', 0.5),
 ('GradientBoostingClassifier', 0.25),
 ('HistGradientBoostingClassifier', 0.0),
 ('KNeighborsClassifier', 0.5),
 ('LinearDiscriminantAnalysis', 0.75),
 ('LinearSVC', 1.0),
 ('LogisticRegression', 1.0),
 ('LogisticRegressionCV', 0.75),
 ('MLPClassifier', 0.5),
 ('MultinomialNB', 1.0),
 ('NearestCentroid', 1.0),
 ('NuSVC', 1.0),
 ('PassiveAggressiveClassifier', 1.0),
 ('Perceptron', 1.0),
 ('QuadraticDiscriminantAnalysis', 0.5),
 ('RandomForestClassifier', 0.75),
 ('RidgeClassifier', 0.75),
 ('RidgeClassifierCV', 0.75),
 ('SGDClassifier', 1.0),
 ('SVC', 0.0)]

In [12]:
from sklearn.linear_model import SGDClassifier

In [13]:
model_sgd = SGDClassifier()
model_sgd.fit(train_input,train_target)

SGDClassifier()

In [14]:
model_sgd.score(test_input,test_target)

0.75

In [15]:
test_list = os.listdir(path_test)
test_list

['en-1.txt',
 'en-2.txt',
 'fr-3.txt',
 'fr-4.txt',
 'id-5.txt',
 'id-6.txt',
 'tl-7.txt',
 'tl-8.txt']

# 테스트 데이터로 검증

In [16]:
for file in test_list:
    alphabet_data = count_alphabet(file,path_test)
    arr_data = np.array(alphabet_data).reshape(1,26)
    print(model_sgd.predict(arr_data))

['en']
['id']
['fr']
['id']
['id']
['id']
['tl']
['id']
